In [1]:
pip install pulp

Note: you may need to restart the kernel to use updated packages.


In [14]:
import pandas as pd
import pulp
import numpy as np
import requests
from io import StringIO

def optimal_redistricting_no_shapefile(county_data, adjacency_matrix, ideal_population, pop_deviation_tolerance=0.10): # Added tolerance
    """Solves redistricting, county-based, no shapefile, with flexible population deviation."""

    num_counties = len(county_data)
    num_districts = 10  # Fixed number of districts

    prob = pulp.LpProblem("Redistricting Problem", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("County_District", (range(num_counties), range(num_districts)), cat='Binary')

    # Objective function (minimize population deviation - now weighted)
    prob += pulp.lpSum([abs(county_data['pop2024'][i] - ideal_population) * x[i][j] for i in range(num_counties) for j in range(num_districts)])

    # Constraints:

    # 1. Each county must be assigned to exactly one district
    for i in range(num_counties):
        prob += pulp.lpSum([x[i][j] for j in range(num_districts)]) == 1

    # 2. Relaxed population balance (using tolerance)
    for j in range(num_districts):
        lower_bound = ideal_population * (1 - pop_deviation_tolerance)
        upper_bound = ideal_population * (1 + pop_deviation_tolerance)
        prob += pulp.lpSum([county_data['pop2024'][i] * x[i][j] for i in range(num_counties)]) >= lower_bound
        prob += pulp.lpSum([county_data['pop2024'][i] * x[i][j] for i in range(num_counties)]) <= upper_bound

    # 3. Contiguity (using adjacency matrix)
    for j in range(num_districts):
        for i in range(num_counties):
            for k in range(num_counties):
                if adjacency_matrix[i, k] == 1 and i != k:
                    prob += x[i][j] <= pulp.lpSum([x[l][j] for l in range(num_counties) if adjacency_matrix[k, l] == 1])

    prob.solve()

    if prob.status == pulp.LpStatusOptimal:
        print("Optimal redistricting found.")
        new_county_data = county_data.copy()
        new_county_data['district'] = 0

        for i in range(num_counties):
            for j in range(num_districts):
                if x[i][j].varValue == 1:
                    new_county_data.loc[i, 'district'] = j + 1

        return new_county_data, prob.objective.value()

    else:
        print("No optimal redistricting found. Status:", pulp.LpStatus[prob.status])
        return None, None

# 1. Load county data (including population) from Excel.
try:
    county_data = pd.read_excel("C:/Users/evehyh/Downloads/Washington County Data.xlsx", sheet_name="counties-table")  # Replace with your Excel file path
except FileNotFoundError:
    print("Population data file not found. Please provide the correct path.")
    exit()

# 2. Load county adjacency data from the URL.
url = "https://www2.census.gov/geo/docs/reference/county_adjacency.txt"
response = requests.get(url)
response.raise_for_status()

county_adjacency_data = pd.read_csv(StringIO(response.text), sep="\t", header=None)
county_adjacency_data.columns = ['County1_Name', 'FIPS1', 'County2_Name', 'FIPS2']

# 3. Create adjacency matrix (using FIPS codes from the Excel file).
county_fips_list = sorted(county_data['fips'].tolist())  # Use 'fips' column from Excel
num_counties = len(county_fips_list)
adjacency_matrix = np.zeros((num_counties, num_counties), dtype=int)

for i, county_fips_i in enumerate(county_fips_list):
    for j, county_fips_j in enumerate(county_fips_list):
        if i != j:
            is_adjacent = county_adjacency_data[
                ((county_adjacency_data['FIPS1'] == county_fips_i) & (county_adjacency_data['FIPS2'] == county_fips_j)) |
                ((county_adjacency_data['FIPS1'] == county_fips_j) & (county_adjacency_data['FIPS2'] == county_fips_i))
            ].any().any()

            if is_adjacent:
                adjacency_matrix[i, j] = 1

# 4. Set ideal population and number of districts (dynamically).
state_population = county_data['pop2024'].sum()
# If you want a fixed number of districts:
#num_districts = 10  # Or whatever number you want.
# OR, to calculate the number of districts based on population and a desired district size:
desired_district_population = 750000 # Example: 750,000 people per district
num_districts = int(np.ceil(state_population / desired_district_population)) # Round up to ensure all population is included

ideal_population = state_population / num_districts

# Add a DISTRICT column if it doesn't exist. Initialize to 0.
if 'DISTRICT' not in county_data.columns:
    county_data['DISTRICT'] = 1

# 5. Run redistricting
new_county_data, objective_value = optimal_redistricting_no_shapefile(county_data, adjacency_matrix, ideal_population, pop_deviation_tolerance=0.10) # 10% tolerance

if new_county_data is not None:
    print(new_county_data)
    print("Objective Function Value:", objective_value)
    new_county_data.to_csv("redistricted_counties.csv", index=False)

No optimal redistricting found. Status: Infeasible


C:\Users\evehyh\AppData\Local\anaconda3\Lib\site-packages\pulp\pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
